# Vítej!

Tento dokument obsahuje zdrojové kódy animací ke čtvrté sérii seriálu KSP. Před spouštěním opět nezapomeň Manim importovat spuštěním následujícího řádku!

In [ ]:
from manim import *

## Booleovské operace

In [ ]:
%%manim -v WARNING -qh BooleanOperations


class BooleanOperations(Scene):
    def construct(self):

        circle = Circle(fill_opacity=0.75, color=RED).scale(2).shift(LEFT * 1.5)
        square = Square(fill_opacity=0.75, color=GREEN).scale(2).shift(RIGHT * 1.5)

        group = VGroup(circle, square)

        self.play(Write(group))

        self.play(group.animate.scale(0.5).shift(UP * 1.6))

        union = Union(circle, square, fill_opacity=1, color=BLUE)

        # postupně voláme Union(), Intersection() a Difference()
        for operation, position, name in zip(
            [Intersection, Union, Difference],
            [LEFT * 3.3, ORIGIN, RIGHT * 4.5],
            ["Průnik", "Sjednocení", "Rozdíl"],
        ):
            result = operation(circle, square, fill_opacity=1, color=DARK_BLUE)
            result_position = DOWN * 1.3 + position
            
            label = Tex(name).move_to(result_position).scale(0.8)
            
            self.play(
                AnimationGroup(
                    FadeIn(result),
                    result.animate.move_to(result_position),
                    FadeIn(label),
                    lag_ratio=0.5,
                )
            )

## Vlastní objekty

In [ ]:
%%manim -v WARNING -qh StackExample


class Stack(VMobject):
    def __init__(self, size, *args, **kwargs):
        # inicializace VGroup objektu
        super().__init__(**kwargs)

        self.squares = VGroup()
        self.labels = VGroup()
        self.index = 0
        self.pointer = Arrow(ORIGIN, UP * 1.2)

        for _ in range(size):
            self.squares.add(Square(side_length=0.8))

        self.squares.arrange(buff=0.15)

        self.pointer.next_to(self.squares[0], DOWN)
        self.add()

        # DŮLEŽITÉ - přidáme do objektu všechny podobjekty!
        self.add(self.squares, self.labels, self.pointer)

    def __get_index_rectangle_color(self):
        """Vrátí barvu aktuálního obdelníků stacku."""
        return self.squares[self.index].get_color()

    def __create_label(self, element):
        """Vytvoření labelu daného prvku (podle barvy a rozměrů čtverců stacku)."""
        return (
            Tex(str(element))
            .scale(self.squares[0].height)
            .set_color(self.__get_index_rectangle_color())
        )

    def push(self, element):
        """Přidá prvek do zásobníku. Vrátí odpovídající animace."""
        self.labels.add(self.__create_label(element).move_to(self.squares[self.index]))
        self.index += 1

        return AnimationGroup(
            FadeIn(self.labels[-1]),
            self.pointer.animate.next_to(self.squares[self.index], DOWN),
            Indicate(
                self.squares[self.index - 1], color=self.__get_index_rectangle_color()
            ),
        )

    def pop(self):
        """Odebere prvek ze zásobníku. Vrátí odpovídající animace."""
        label = self.labels[-1]
        self.labels.remove(label)
        self.index -= 1

        return AnimationGroup(
            FadeOut(label),
            self.pointer.animate.next_to(self.squares[self.index], DOWN),
            Indicate(
                self.squares[self.index],
                color=self.__get_index_rectangle_color(),
                scale_factor=1 / 1.2,
            ),
        )

    def clear(self):
        """Vyčistí zásobník. Vrátí odpovídající animaci."""
        result = AnimationGroup(*[self.pop() for _ in range(self.index)], lag_ratio=0)

        self.index = 0

        return result


class StackExample(Scene):
    def construct(self):
        stack = Stack(10)

        # na objekt fungují správně libovolné animace
        self.play(Write(stack))

        self.wait(0.5)

        for i in range(5):
            self.play(stack.push(i))

        self.play(stack.pop())

        self.wait(0.5)

        # můžeme používat i animate syntax!
        # měnění barvy se rekurzivně aplikuje na všechny podobjekty
        self.play(stack.animate.scale(1.3).set_color(BLUE))

        self.wait(0.5)

        for i in range(2):
            self.play(stack.push(i))

        self.play(stack.pop())

        self.play(stack.clear())

        self.play(FadeOut(stack))

## Jiné grafy

In [ ]:
%%manim -v WARNING -qh GraphExample

from math import sin


class GraphExample(Scene):
    def construct(self):
        # osy - rozmezí a značení os x, y
        axes = Axes(x_range=[-5, 5], y_range=[-3, 7])
        labels = axes.get_axis_labels(x_label="x", y_label="y")

        def f1(x):
            return x ** 2

        def f2(x):
            return sin(x)

        # objekty vykreslených funkcí
        g1 = axes.plot(f1, color=RED)
        g2 = axes.plot(f2, color=BLUE)

        self.play(Write(axes), Write(labels))

        self.play(AnimationGroup(Write(g1), Write(g2), lag_ratio=0.5))

        self.play(Unwrite(axes), Unwrite(labels), Unwrite(g1), Unwrite(g2))

In [ ]:
%%manim -v WARNING -qh DiscontinuousGraphExample

from math import sin


class DiscontinuousGraphExample(Scene):
    def construct(self):
        axes = Axes(x_range=[-5, 5], y_range=[-3, 7])
        labels = axes.get_axis_labels(x_label="x", y_label="y")

        def f(x):
            return 1 / x

        g_bad = axes.plot(f, color=RED)

        # rozdělení na dvě části podle hodnot x
        g_left = axes.plot(f, x_range=[-5, -0.1], color=GREEN)
        g_right = axes.plot(f, x_range=[0.1, 5], color=GREEN)

        self.play(Write(axes), Write(labels))

        self.play(Write(g_bad))
        self.play(FadeOut(g_bad))

        self.play(AnimationGroup(Write(g_left), Write(g_right), lag_ratio=0.5))

        self.play(Unwrite(axes), Unwrite(labels), Unwrite(g_left), Unwrite(g_right))

In [ ]:
%%manim -v WARNING -qh ParametricGraphExample

from math import sin, cos


class ParametricGraphExample(Scene):
    def construct(self):
        axes = Axes(x_range=[-10, 10], y_range=[-5, 5])
        labels = axes.get_axis_labels(x_label="x", y_label="y")

        def f1(t):
            """Parametrická funkce kružnice."""
            return (cos(t) * 3 - 4.5, sin(t) * 3)

        def f2(t):
            """Parametrická funkce <3."""
            return (
                0.2 * (16 * (sin(t)) ** 3) + 4.5,
                0.2 * (13 * cos(t) - 5 * cos(2 * t) - 2 * cos(3 * t) - cos(4 * t)),
            )

        # objekty vykreslených funkcí
        # místo axes.plot používáme axes.plot_parametric_curve
        # parametr t_range určuje, jaké je rozmezí parametru t
        g1 = axes.plot_parametric_curve(f1, color=RED, t_range=[0, 2 * PI])
        g2 = axes.plot_parametric_curve(f2, color=BLUE, t_range=[-PI, PI])

        self.play(Write(axes), Write(labels))

        self.play(AnimationGroup(Write(g1), Write(g2), lag_ratio=0.5))

        self.play(Unwrite(axes), Unwrite(labels), Unwrite(g1), Unwrite(g2))

In [ ]:
%%manim -v WARNING -qh LineGraphExample

from random import random, seed


class LineGraphExample(Scene):
    def construct(self):
        seed(0xDEADBEEF2)  # hezčí hodnoty :P

        # hodnoty ke grafování (x a y)
        # u np.arange(l, r, step) vrátí pole hodnot od l do r (nevčetně) s kroky velikosti step
        x_values = np.arange(-1, 1 + 0.25, 0.25)
        y_values = [random() for _ in x_values]

        # osy (tentokrát s čísly)
        axes = Axes(
            x_range=[-1, 1, 0.25],
            y_range=[-0.1, 1, 0.25],
            # nastavení čísel - hodnoty a počet desetinných míst
            x_axis_config={"numbers_to_include": x_values},
            y_axis_config={"numbers_to_include": np.arange(0, 1, 0.25)},
            axis_config={"decimal_number_config": {"num_decimal_places": 2}},
        )

        labels = axes.get_axis_labels(x_label="x", y_label="y")

        # místo axes.plot používáme axes.plot_line_graph
        graph = axes.plot_line_graph(x_values=x_values, y_values=y_values)

        self.play(Write(axes), Write(labels))

        self.play(Write(graph), run_time=2)

        self.play(Unwrite(axes), Unwrite(labels), Unwrite(graph))

## 3D operace

In [ ]:
%%manim -v WARNING -qh Axes3DExample


class Axes3DExample(ThreeDScene):
    def construct(self):
        # 3D osy
        axes = ThreeDAxes()

        x_label = axes.get_x_axis_label(Tex("x"))
        y_label = axes.get_y_axis_label(Tex("y")).shift(UP * 1.8)

        # 3D varianta Dot() objektu
        dot = Dot3D()

        # zmenšení zoomu, abychom viděli osy
        self.set_camera_orientation(zoom=0.5)

        self.play(FadeIn(axes), FadeIn(dot), FadeIn(x_label), FadeIn(y_label))

        self.wait(0.5)

        # animace posunutí kamery tak, aby byly osy dobře vidět
        self.move_camera(phi=75 * DEGREES, theta=30 * DEGREES, zoom=1, run_time=1.5)

        # vestavěný updater, který kameru začne rotovat (aby na scénu bylo lépe vidět)
        self.begin_ambient_camera_rotation(rate=0.15)

        # jedna tečka za každý směr
        upDot = dot.copy().set_color(RED)
        rightDot = dot.copy().set_color(BLUE)
        outDot = dot.copy().set_color(GREEN)

        self.wait(1)

        self.play(
            upDot.animate.shift(UP),
            rightDot.animate.shift(RIGHT),
            outDot.animate.shift(OUT),
        )

        self.wait(2)

In [ ]:
%%manim -v WARNING -qh Rotation3DExample


class Rotation3DExample(ThreeDScene):
    def construct(self):
        # přidání jednoduché základní krychle
        cube = Cube(side_length=3, fill_opacity=1)

        self.begin_ambient_camera_rotation(rate=0.3)

        # posunutí orientace kamery bez animace
        self.set_camera_orientation(phi=75 * DEGREES, theta=30 * DEGREES)

        self.play(Write(cube), run_time=2)

        self.wait(3)

        self.play(Unwrite(cube), run_time=2)

In [ ]:
%%manim -v WARNING -qh Basic3DExample


class Basic3DExample(ThreeDScene):
    def construct(self):
        # přidání jednoduché základní krychle
        cube = Cube(side_length=3, fill_opacity=0.5)

        self.set_camera_orientation(phi=75 * DEGREES, theta=30 * DEGREES)

        self.play(FadeIn(cube))

        for axis in [RIGHT, UP, OUT]:
            self.play(Rotate(cube, PI / 2, about_point=ORIGIN, axis=axis))

        self.play(FadeOut(cube))

## Kostry úloh

### Simulace binomického rozložení [6b]

In [ ]:
%%manim -v WARNING -qh BinomialDistributionSimulation


class BinomialDistributionSimulation(Scene):
    def construct(self):
        pass

In [ ]:
%%manim -v WARNING -qh BezierExample

from random import choice, seed


class MoveAndFade(Animation):
    def __init__(self, mobject: Mobject, path: VMobject, **kwargs):
        self.path = path
        self.original = mobject.copy()
        super().__init__(mobject, **kwargs)

    def interpolate_mobject(self, alpha: float) -> None:
        point = self.path.point_from_proportion(self.rate_func(alpha))

        # tohle není úplně čisté, jelikož pokaždé vytváříme nový objekt
        # je to kvůli tomu, že obj.fade() nenastavuje průhlednost ale přidává jí
        self.mobject.become(self.original.copy()).move_to(point).fade(alpha)


class BezierExample(Scene):
    def construct(self):
        # křivku definujeme přes čtyři body:
        # 2 krajní, ve kterých začíná a končí
        # 2 kontrolní, které určují tvar
        positions = [
            UP + LEFT * 3,  # počáteční
            UP + RIGHT * 2,  # 1. kontrolní
            DOWN + LEFT * 2,  # 2. kontrolní
            DOWN + RIGHT * 3,  # koncový
        ]

        points = VGroup(*[Dot().move_to(position) for position in positions]).scale(1.5)

        # rozlišíme kontrolní body
        points[1].set_color(BLUE)
        points[2].set_color(BLUE)

        bezier = CubicBezier(*positions).scale(1.5)

        self.play(Write(bezier), Write(points))

        # animace posunu
        circle = Circle(fill_opacity=1, stroke_opacity=0).scale(0.25).move_to(points[0])

        self.play(FadeIn(circle, shift=RIGHT * 0.5))
        self.play(MoveAlongPath(circle, bezier))

        self.play(FadeOut(circle))

        # animace posunu s mizením
        circle = (
            Circle(fill_color=GREEN, fill_opacity=1, stroke_opacity=0)
            .scale(0.25)
            .move_to(points[0])
        )

        self.play(FadeIn(circle, shift=RIGHT * 0.5))
        self.play(MoveAndFade(circle, bezier))

        self.play(FadeOut(bezier), FadeOut(points), FadeOut(circle))

### 3D Game of Life [9b]

#### Dvoustavový [5b]

In [ ]:
%%manim -v WARNING -qh GOLTwoState


class GOLTwoState(ThreeDScene):
    def construct(self):
        pass

#### Vícestavový [3b]

In [ ]:
%%manim -v WARNING -qh GOLMultiState


class GOLMultiState(ThreeDScene):
    def construct(self):
        pass

#### Vlastní [1b]

In [ ]:
%%manim -v WARNING -qh GOLCustom


class GOLCustom(ThreeDScene):
    def construct(self):
        pass